In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import pickle


In [16]:
df= pd.read_csv('kickstarter2.csv')

In [17]:


df.set_index(['month_created'], inplace=True)
df['status']=df['status'].map({'successful':1, 'failed':0 }).astype(float)
df['staff_pick']=df['staff_pick'].astype(int)
df['spotlight']=df['spotlight'].astype(int)

In [18]:
target = 'status'
y = df[target]
X = df.drop(columns=target)

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
XtrainEn = X_train.select_dtypes(include=[object])
X_valEn = X_val.select_dtypes(include=[object])

In [21]:
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(XtrainEn)
enc.fit(X_valEn)

X_trainOH = enc.transform(XtrainEn).toarray()
X_valOH = enc.transform(X_valEn).toarray()


In [22]:
X_valencoded = pd.DataFrame(X_valOH)
X_trainencoded = pd.DataFrame(X_trainOH)
 

In [23]:
X_trainencoded['index'] = range(1, len(X_trainencoded)+1)
X_train['index'] = range(1, len(X_train)+1)
X_valencoded['index'] = range(1, len(X_valencoded)+1)
X_val['index'] = range(1, len(X_val)+1)
X_train.set_index(['index'], inplace=True)
X_val.set_index(['index'], inplace=True)
X_trainencoded.set_index(['index'], inplace=True)
X_valencoded.set_index(['index'], inplace=True)
X_train.drop('country_code', axis=1, inplace=True)
X_train.drop('category', axis=1, inplace=True)
X_train.drop('currency_code', axis=1, inplace=True)
X_val.drop('country_code', axis=1, inplace=True)
X_val.drop('category', axis=1, inplace=True)
X_val.drop('currency_code', axis=1, inplace=True)
X_train_new=pd.concat([X_train, X_trainencoded],axis=1)
X_val_new=pd.concat([X_val, X_valencoded], axis=1)



In [24]:

 scaler=MaxAbsScaler()
 X_train_scaled = scaler.fit_transform(X_train_new)
 X_val_scaled = scaler.fit_transform(X_val_new)

C:\Users\O_O_O\.virtualenvs\Datum-7gjrdNBa\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\O_O_O\.virtualenvs\Datum-7gjrdNBa\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\O_O_O\.virtualenvs\Datum-7gjrdNBa\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\O_O_O\.virtualenvs\Datum-7gjrdNBa\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An er

In [30]:
grb_classifier = GradientBoostingClassifier(n_estimators=40, learning_rate=0.5, max_features=3, max_depth=6, random_state=0)
grb_model=grb_classifier.fit(X_train_scaled, y_train)

y_pred_grb = grb_model.predict(X_val_scaled)

print(classification_report(y_val, y_pred_grb, digits=3))


              precision    recall  f1-score   support

         0.0      0.988     0.924     0.955     15589
         1.0      0.952     0.993     0.972     23590

    accuracy                          0.965     39179
   macro avg      0.970     0.958     0.963     39179
weighted avg      0.966     0.965     0.965     39179



In [26]:

with open('grb_pickle', 'wb') as f:
    pickle.dump(grb_model, f)


In [ ]:
def input_wrangle():
    